In [19]:
#export
import pytex.core.BinaryOperator as BinaryOperator
from pytex.helpers import varArgFunc
from pytex.core.Precedence import PRECEDENCE

class Var():
    def __init__(self, name, backward_operation=None):
        self._name, self._backward_operation = name, backward_operation
    
    @property
    def backward_operation(self):
        return self._backward_operation
    
    @property
    def precedence(self):
        return (PRECEDENCE.VAR 
                if self._backward_operation is None 
                else self._backward_operation.precedence)
    
    def build(self):
        return (self._name 
                if self._backward_operation is None 
                else self._backward_operation.build())
    
    def binary_operation(self, other, operator):
        operation = BinaryOperation(operator)
        other = makeVar(other)
        output = Var('output', operation(self, other))
        return output
    
    def __add__(self, other):
        return self.binary_operation(other, BinaryOperator.Plus)
    def __sub__(self, other):
        return self.binary_operation(other, BinaryOperator.Minus)
    def __mul__(self, other):
        return self.binary_operation(other, BinaryOperator.Multiply)
    def __pow__(self, other):
        return self.binary_operation(other, BinaryOperator.Power)
    def __or__(self, other):
        return self.binary_operation(other, BinaryOperator.Space)
    
    def lt(self, other):
        return self.binary_operation(other, BinaryOperator.LessThan)
    def __lt__(self, other):
        return self.lt(other)
    
    def gt(self, other):
        return self.binary_operation(other, BinaryOperator.GreaterThan)
    def __gt__(self, other):
        return self.gt(other)
    
    def le(self, other):
        return self.binary_operation(other, BinaryOperator.LessThanEqual)
    def __le__(self, other):
        return self.le(other)
    
    def ge(self, other):
        return self.binary_operation(other, BinaryOperator.GreaterThanEqual)
    def __ge__(self, other):
        return self.ge(other)
    
    def eq(self,other):
        return self.binary_operation(other, BinaryOperator.Equal)
    def ne(self,other):
        return self.binary_operation(other, ~BinaryOperator.Equal)
    def __eq__(self, other):
        self.eq(other)
    def __ne__(self, other):
        self.ne(other)
    
#     def __neg__(self):
#         return self.apply(NegateFunc(), self)
#     def root(self, other):
#         return self.apply(RootFunc(), self, other)
#     def __truediv__(self, other):
#         return self.apply(FracFunc(), self, other)
    
def isVar(a):
    return isinstance(a, Var)

def makeVar(*args):
    return varArgFunc(lambda arg: arg if isVar(arg) else Var(str(arg)), *args)

def addEnv(s, env='gather*'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env

In [26]:
#export
from pytex.LatexInterface import create_latex_for_binary_operator, surround_with_parens

class Operation():
    def __init__(self, operator):
        self._operator = operator
    
    @property
    def precedence(self):
        return self.operator.precedence
    
    def __call__(self, *args):
        raise NotImplementedError()
    
    def surround_if_needed(self, var_instance):
        return (self.get_surrounded_with_parens(var_instance.build())
               if self.should_surround_with_parens(var_instance)
               else var_instance.build())
    
    def get_surrounded_with_parens(self, string):
        return surround_with_parens(string)
    
    def should_surround_with_parens(self, var_instance):
        return (var_instance.backward_operation is not None and 
                var_instance.backward_operation.precedence < self.precedence)   
    
    
class BinaryOperation(Operation):
    def __init__(self, operator):
        super().__init__(operator)
        self._arg1, self._arg2 = None, None
        
    def __call__(self, arg1, arg2):
        self._arg1, self._arg2 = arg1, arg2
        return self
    
    def build(self):
        assert self._arg1 is not None and self._arg2 is not None, 'Call the Class to set arguments'
        arg1, arg2 = self.surround_if_needed(self._arg1), self.surround_if_needed(self._arg2)
        return create_latex_for_binary_operator(self._operator, arg1, arg2)
    
    def __invert__(self):
        self._operator.__invert__()
        return self

In [30]:
! ../notebook2script.py Ast.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/core/Ast.ipynb to /home/default/Documents/py-latex/pytex/pytex/core/Ast.py


In [27]:
from pytex.platforms.jupyter import latex
a, b, c = makeVar('a','b', 'c')
latex(a + b)

<IPython.core.display.Latex object>